<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-01-22 14:20:06
-------------------
qualified stocks: 85
with latest results: 79
still star stocks: 43
-------------------
Initial Investment:  1.04 C
CY Investment:  1.17 C
Reserve:  4.22 L
Current:  1.14 C
-------------------
Today PnL: -1.29 L (-1.12%)
Current PnL: -16.51 L (-14.07%)
CY Booked + Current PnL: -3.24 L (-2.76%)
-------------------
Total profit:  2.00 L
Total loss:  -18.51 L
-------------------
Total Booked + Current PnL: 9.87 L (9.46%)
Total Booked PnL: 26.38 L (25.3%)
Curr Year Booked PnL: 13.27 L (11.61%)
Prev Year Booked PnL: 13.11 L (12.57%)
Est FTT:  1.85 C
Est FTT PnL: 70.57 L (61.76%)
-------------------
Est LTT:  2.44 C
Est LTT PnL: 1.30 C (113.47%)
Deployed:  1.04 C
Current:  1.14 C
CAGR/XIRR %: 7.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
65   SBILIFE  1483.67  1928.65    -9.78       H-LC  89.41   99243.0   
76       TCS  4164.02  4998.00     2.36       X-LC  72.94  161850.0   
69       SIS   477.00   528.00    56.94       M-SC  22.35   56593.0   
31   HAVELLS  1588.50  2077.12    -2.44       X-LC  40.59   96966.0   
9   AXISBANK   986.10  1317.30   -33.00       H-LC  17.65   97731.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
65      -1647.0  31907.0       -0.48         -1.63  32.15  29.99   NaN   
76       -547.0  33066.0        2.83         -0.34  20.43  20.03  15.0   
69     -24497.0  33169.0        0.76        -30.21  58.61  10.69 -27.0   
31      -3110.0  33890.0       -2.12         -3.11  34.95  30.76   NaN   
9       -2851.0  36630.0       -1.24         -2.83  37.48  33.59   NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
65    -0.05        0.90     9.01       AR      ATH   INSURANCE  
76    -0.02        1.47    13.86      X40      BTT          IT  
69    -0.74        0.51     0.76     X40N      NTT        MISC  
31    -0.09        0.88    20.25      X40      ATH  ELECTRICAL  
9     -0.08        0.89     0.00      X40      ATH       BANKS

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
38        INFY  1436.24  2275.00     4.10       X-LC  63.53  193643.0   
76         TCS  4164.02  4998.00     2.36       X-LC  72.94  161850.0   
25  EASEMYTRIP    18.68    26.40   -17.18       M-SC   4.71  125425.0   
45        LTIM  5664.39  7262.39    11.24       X-LC  83.53  175776.0   
32    HDFCBANK  1549.96  2170.00   -10.96       X-LC  36.47  348390.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
38      44274.0   42950.0        3.40         29.64  22.18  58.40  37.0   
76       -547.0   33066.0        2.83         -0.34  20.43  20.03  15.0   
25     -40976.0  109747.0        2.55        -24.63  87.50  41.33 -93.0   
45       5844.0   42098.0        1.75          3.44  23.95  28.21   1.0   
32      22898.0  107304.0        1.01          7.04  30.80  40.00   4.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
38     1.03        1.76    35.12      X40      BTT       IT  
76    -0.02        1.47    13.86      X40      BTT       IT  
25    -0.37        1.14     2.55     XY24      NTT   TRAVEL  
45     0.14        1.60    29.12     X200      ATH       IT  
32     0.21        3.17    21.50      X40      BTT    BANKS

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
73     TANLA   991.56  2007.31   -44.66       H-SC  49.41  138291.0   
77  TITAGARH  1117.46  1548.00    -9.37       M-SC   1.18  163155.0   
1   AARTIIND   487.04   919.00    -1.65       M-SC  92.94   72244.0   
30   HATHWAY    19.74    31.40    11.95       L-SC  42.35   61000.0   
70  SONACOMS   578.05  1006.00   -28.05       M-MC  29.41   93616.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  XIRR  \
73     -80844.0  305319.0       -6.05        -36.89  220.78  102.44 -31.0   
77     -18991.0   89164.0       -5.15        -10.43   54.65   38.53 -29.0   
1       -9092.0   81231.0       -4.51        -11.18  112.44   88.69 -11.0   
30     -17960.0   64599.0       -3.91        -22.75  105.90   59.07 -20.0   
70      -7543.0   82429.0       -3.86         -7.46   88.05   74.03  -2.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy       Category  
73    -0.26        1.26     0.00       AR      ATH             IT  
77    -0.21        1.48    21.56     XY24      NTT    ENGINEERING  
1     -0.11        0.66     8.73       XR      NTT      CHEMICALS  
30    -0.28        0.55     4.88       XR      NTT  ENTERTAINMENT  
70    -0.09        0.85     0.00       AR      BTT           AUTO

In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
46    LUXIND  1448.19  2442.0    26.11       M-SC  10.59  158916.0   
27     GICRE   335.06   583.0     7.26       H-MC  15.29  184567.0   
64  SBIETFPB   241.24   400.0      NaN        NaN  62.35  183353.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
46      21338.0   73070.0       -1.11         15.51  45.98  68.62   11.0   
27      38146.0   70209.0       -2.87         26.05  38.04  74.00  110.0   
64        734.0  119454.0       -0.14          0.40  65.15  65.81    1.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
46     0.29        1.44    55.85     X40N      NTT   TEXTILES  
27     0.54        1.68    36.51     XY24      BTT  INSURANCE  
64     0.01        1.67     6.22     XY24      BTT      BANKS

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT   Dev%_PE Conviction    RSP  Current  \
61  SAMMAANCAP   170.35   326.0   -182.05       M-SC  96.47  96144.0   
70    SONACOMS   578.05  1006.0    -28.05       M-MC  29.41  93616.0   
1     AARTIIND   487.04   919.0     -1.65       M-SC  92.94  72244.0   
13   BANKINDIA   112.90   190.0    -33.87       M-MC  54.12  86749.0   
6   ASIANTILES    75.41   137.0 -12436.67       M-SC  30.59  76581.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
61      -6066.0  99451.0       -0.42         -5.93  103.44  91.37   NaN   
70      -7543.0  82429.0       -3.86         -7.46   88.05  74.03  -2.0   
1       -9092.0  81231.0       -4.51        -11.18  112.44  88.69 -11.0   
13     -13506.0  81969.0       -2.37        -13.47   94.49  68.29 -15.0   
6      -17229.0  93850.0       -2.99        -18.37  122.55  81.67 -14.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
61    -0.06        0.87    20.35       XR      NTT    FINANCE  
70    -0.09        0.85     0.00       AR      BTT       AUTO  
1     -0.11        0.66     8.73       XR      NTT  CHEMICALS  
13    -0.16        0.79     8.12       XR      NTT      BANKS  
6     -0.18        0.70    19.30       XR      NTT   CERAMICS

In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  Current P/L  \
27  GICRE   335.06   583.0     7.26       H-MC  15.29  184567.0      38146.0   
38   INFY  1436.24  2275.0     4.10       X-LC  63.53  193643.0      44274.0   

    FTT Amt  Today P/L%  Current P/L%   FTT%  OTT%   XIRR  RRR Ind  \
27  70209.0       -2.87         26.05  38.04  74.0  110.0     0.54   
38  42950.0        3.40         29.64  22.18  58.4   37.0     1.03   

    CurrAlloc%  Gained% Criteria Strategy   Category  
27        1.68    36.51     XY24      BTT  INSURANCE  
38        1.76    35.12      X40      BTT         IT

In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction     RSP   Current  \
20   COFFEEDAY    59.14    80.0  -128.68       L-SC  100.00   52454.0   
10  BAJAJFINSV  1473.47  2392.0   -13.84       X-LC   98.82  135057.0   
63     SBICARD   765.91  1075.0   -28.22       H-MC   97.65  176556.0   
61  SAMMAANCAP   170.35   326.0  -182.05       M-SC   96.47   96144.0   
11  BAJFINANCE  7055.29  9150.0   -28.01       X-LC   95.29  226041.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
20     -61095.0  101147.0       -0.00        -53.80  192.83  35.27   NaN   
10      20126.0   51524.0        0.84         17.51   38.15  62.34   8.0   
63      -3433.0   76078.0       -1.48         -1.91   43.09  40.36  -4.0   
61      -6066.0   99451.0       -0.42         -5.93  103.44  91.37   NaN   
11       7327.0   57618.0       -2.00          3.35   25.49  29.69  15.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
20    -0.60        0.48    20.78       XR      NTT   HOTELS  
10     0.39        1.23    17.28      X40      BTT  FINANCE  
63    -0.05        1.60    13.17     XY24      NTT  FINANCE  
61    -0.06        0.87    20.35       XR      NTT  FINANCE  
11     0.13        2.05    16.11      X40      BTT  FINANCE

In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.99
1     25       42.62
2     50       73.95

In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    41.71
X40     19.31
X40N    11.84
AR      10.98
XR       9.48
XY25     3.65
X200     1.60
SR       1.44
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
M-SC    25.87
X-LC    23.94
H-SC    14.63
M-MC     9.68
X-MC     6.36
X-SC     4.70
H-MC     4.64
H-LC     3.26
L-SC     2.72
M-LC     2.54
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FINANCE             12.96        -14.48   62.69
BANKS               12.53        -19.90   79.67
FMCG                 9.81         -5.18   42.60
IT                   9.15        -10.67   66.29
PAINTS               6.13        -21.88   57.95
MISC                 5.80        -18.57   56.18
INSURANCE            5.43          2.18   46.55
CHEMICALS            4.19        -17.31   94.62
AUTO                 3.31        -12.89   62.52
FOOTWEAR             3.00        -25.01   66.32
ELECTRICAL           2.88         -4.43   39.72
HEALTHCARE           2.86         -7.15   44.05
POWER                2.32        -10.73   55.02
TRAVEL               2.22        -25.78   77.13
TEXTILES             2.14          2.53   48.66
CERAMICS             2.05        -26.39   77.80
TYRES                2.02         -2.36   35.35
REFINERIES           1.83         -1.01   19.62
STEEL                1.66        -23.50  115.54
JEWELLERY            1.57        -66.85  119.61
ENGINEERING          1.48        -11.64   54.65
CEMENT               1.32        -29.81   96.60
ENTERTAINMENT        1.13        -30.89   96.70
DURABLES             0.88         -3.52   54.65
AC                   0.86        -34.97  109.47
HOTELS               0.48       -116.47  192.83

In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3068364.0     30
XR        1047514.0     13
X40N       965008.0     14
AR         841741.0     10
X40        815632.0     13
SR         187201.0      2
XY25        89413.0      2
X200        42098.0      1

In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
M-SC        2148997.0     26
H-SC        1590457.0     12
X-LC         876389.0     15
M-MC         873031.0      9
X-SC         385402.0      5
L-SC         297620.0      4
X-MC         281107.0      5
H-MC         212048.0      3
H-LC         142466.0      3
M-LC         130000.0      2

In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
M-SC       XY24      996731.0     10
H-SC       XY24      751689.0      5
M-MC       XY24      660525.0      6
X-LC       X40       583855.0      9
M-SC       XR        458073.0      6
           X40N      433560.0      6
H-SC       X40N      345765.0      4
           AR        305319.0      1
H-MC       XY24      212048.0      3
H-SC       XR        187684.0      2
L-SC       XR        165746.0      2
X-LC       AR        161023.0      3
M-MC       XR        130077.0      2
M-LC       XY24      130000.0      2
M-SC       AR        126303.0      2
X-SC       SR        121570.0      1
H-LC       X40       110559.0      2
X-SC       XR        105934.0      1
X-MC       X40N       95460.0      2
X-SC       X40N       90223.0      2
X-LC       XY25       89413.0      2
M-MC       AR         82429.0      1
X-MC       AR         78618.0      1
L-SC       XY24       75732.0      1
M-SC       X40        68699.0      1
X-SC       XY24       67675.0      1
M-SC       SR         65631.0      1
L-SC       AR         56142.0      1
X-MC       XY24       54510.0      1
           X40        52519.0      1
X-LC       X200       42098.0      1
H-LC       AR         31907.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
